In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.datasets import make_blobs
from cvxopt import matrix, solvers

In [ ]:
def create_synthetic_data():
  X, y = make_blobs(n_samples = 40, centers=[(0, 3), (3, 0)], cluster_std=[0.5, 0.5], random_state=6)
  idx_cluster_0 = np.where(y == 0)
  y[idx_cluster_0] = -1
  X = X.T
  return X, y 

In [ ]:
X,y = create_synthetic_data()

In [ ]:
class SVM:
  def __init__(self, X, y):
    self.X = X
    self.y = y
    

  def fit(self):
    self.V = np.zeros((self.X.shape[0], self.X.shape[1]))
    for i in range(self.X.shape[1]):
      self.V[:,i] = self.X[:,i] * y[i]
    K =self.V.T.dot(self.V)
    P = matrix(K, tc="d")
    q = matrix(-np.ones((K.shape[0], 1)), tc="d")
    G = matrix(-np.eye((K.shape[0])), tc="d")
    h = matrix(np.zeros((K.shape[0],1)),tc="d")
    A = matrix(np.array([self.y]),tc="d")
    b = matrix(np.zeros((1, 1)), tc="d") 
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)
    self.lambda_ = np.array(sol['x'])
    # derive weight and bias 
    epsilon = 1e-6
    S = np.where(self.lambda_ > epsilon)[0]
    V_S = self.V[:,S]
    lambda_S = self.lambda_[S]
    y_S = np.array([self.y])[:,S]
    self.weight = V_S.dot(lambda_S)
    # print(self.weight.shape)
    self.bias = np.mean(np.array([y_S]).T - self.weight.T.dot(X[:,S]))
    return self.lambda_, self.weight, self.bias


  def predict(self, input):
    input = np.array([input])
    h = self.bias + input.dot(self.weight)
    return np.sign(h).squeeze()

In [ ]:
SVM_v1 = SVM(X,y)
coef = SVM_v1.fit()
coef

In [ ]:
SVM_v1.predict([-5,1])

array(-1.)